### Below code from mrbichel's gist [here](https://gist.github.com/mrbichel/1871929) and adjusted slightly to work (see comment at [the gist page](https://gist.github.com/mrbichel/1871929))

# Python port of Paul Bourke's random strange attractor generator using lyapunov exponents

In [34]:
# Python port of Paul Bourke's http://local.wasp.uwa.edu.au/~pbourke/fractals/lyapunov/gen.c
# By Johan Bichel Lindegaard - http://johan.cc

import math
import random
from PIL import Image, ImageDraw

import os



MAXITERATIONS = 100000
NEXAMPLES = 1000

def createAttractor():
    for n in range(NEXAMPLES):        
        lyapunov = 0
        xmin= 1e32
        xmax=-1e32
        ymin= 1e32
        ymax=-1e32
        ax, ay, x, y = [], [], [], []
        
        # Initialize coefficients for this attractor
        for i in range(6):
            ax.append(random.uniform(-2, 2))
            ay.append(random.uniform(-2, 2))
    
        # Calculate the attractor
        drawit = True;
        x.append(random.uniform(-0.5, 0.5))
        y.append(random.uniform(-0.5, 0.5))
        
        d0 = -1
        while d0 <= 0:
            xe = x[0] + random.uniform(-0.5, 0.5) / 1000.0
            ye = y[0] + random.uniform(-0.5, 0.5) / 1000.0
            dx = x[0] - xe
            dy = y[0] - ye
            d0 = math.sqrt(dx * dx + dy * dy)

        for i in range(MAXITERATIONS):
            # Calculate next term
            
            x.append(ax[0] + ax[1]*x[i-1] + ax[2]*x[i-1]*x[i-1] + ax[3]*x[i-1]*y[i-1] + ax[4]*y[i-1] + ax[5]*y[i-1]*y[i-1])
            y.append(ay[0] + ay[1]*x[i-1] + ay[2]*x[i-1]*x[i-1] + ay[3]*x[i-1]*y[i-1] + ay[4]*y[i-1] + ay[5]*y[i-1]*y[i-1])
            xenew = ax[0] + ax[1]*xe + ax[2]*xe*xe + ax[3]*xe*ye + ax[4]*ye + ax[5]*ye*ye
            yenew = ay[0] + ay[1]*xe + ay[2]*xe*xe + ay[3]*xe*ye + ay[4]*ye + ay[5]*ye*ye

            # Update the bounds 
            xmin = min(xmin,x[i])
            ymin = min(ymin,y[i])
            xmax = max(xmax,x[i])
            ymax = max(ymax,y[i])

            # Does the series tend to infinity
            if xmin < -1e10 or ymin < -1e10 or xmax > 1e10 or ymax > 1e10:
                drawit = False
                print "infinite attractor"
                break

            # Does the series tend to a point
            dx = x[i] - x[i-1]
            dy = y[i] - y[i-1]
            if abs(dx) < 1e-10 and abs(dy) < 1e-10:
                drawit = False
                print "point attractor"
                break
            

            # Calculate the lyapunov exponents
            if i > 1000:
                dx = x[i] - xenew
                dy = y[i] - yenew
                dd = math.sqrt(dx * dx + dy * dy)
                lyapunov += math.log(math.fabs(dd / d0))
                xe = x[i] + d0 * dx / dd
                ye = y[i] + d0 * dy / dd
            
        # Classify the series according to lyapunov
        if drawit:
            if abs(lyapunov) < 10:
                print "neutrally stable"
                drawit = False
            elif lyapunov < 0:
                print "periodic {} ".format(lyapunov)
                drawit = False 
            else:
                print "chaotic {} ".format(lyapunov) 
            
        # Save the image
        if drawit:
            saveAttractor(n,ax,ay,xmin,xmax,ymin,ymax,x,y)

def saveAttractor(n,a,b,xmin,xmax,ymin,ymax,x,y):
    width, height = 500, 500
    
    if not os.path.exists("output"):
        os.makedirs("output")

    # Save the parameters
    with open("output/{}.txt".format(n), "w") as f:
        f.write("{} {} {} {}\n".format(xmin, ymin, xmax, ymax))
        for i in range(6):
            f.write("{} {}\n".format(a[i], b[i]))
        f.close()
    
    # Save the image
    image = Image.new("RGBA", (width, height))
    draw = ImageDraw.Draw(image)
    
    for i in range(MAXITERATIONS):
        ix = width * (x[i] - xmin) / (xmax - xmin)
        iy = height * (y[i] - ymin) / (ymax - ymin)
        if i > 100:
            draw.point([ix, iy], fill="black")
    
    image.save("output/{}.png".format(n), "PNG")
    print "saved attractor to ./output/{}.png".format(n)

createAttractor()

infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
chaotic 633358.465966 
saved attractor to ./output/19.png
infinite attractor
infinite attractor
infinite attractor
infinite attractor
point attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
point attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attractor
infinite attracto

In [7]:
cd output

/home/main/notebooks/demos/output


In [63]:
ls

15.png  170.png  250.png  274.png  600.png  843.png
15.txt  170.txt  250.txt  274.txt  600.txt  843.txt


In [8]:
cd ..

/home/main/notebooks/demos


> above line `cd ..` is just to stay in main working directory to make things easier because otherwise as run the notebook successively it will keep generating `output` directories inside themselves recursively.

### Example image

![image](output/170.png)

The `ls` command seen above list the files generated in the `output` directory. To see the images generated like that above, enter `![title](output/170.png)` in a cell formatted as Markdown, changing `170.png` to your file name

### Saved parameters

In [57]:
cat output/170.txt

-0.291264925485 -0.3939366234 0.115476631678 0.293637073617
0.0574389957312 0.0612484167781
0.0603007268014 1.84804095915
-1.02165397294 0.780092207044
1.54241348196 1.30855118999
-0.491576815391 0.252217963784
-1.45043295758 0.320851455295


The default image from the script is not quite as [nice as PovRay renders](http://news.povray.org/povray.binaries.images/message/%3Cweb.446154956ea18d9a61fef93c0%40news.povray.org%3E/#%3Cweb.446154956ea18d9a61fef93c0%40news.povray.org%3E) but impressively runs in Jupyter notebook/Python in a browser